In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
178,AFG,Asia,Afghanistan,2020-07-06,32951.0,279.0,864.0,38.0,846.453,7.167,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
298,ALB,Europe,Albania,2020-07-06,2893.0,74.0,76.0,2.0,1005.282,25.714,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
482,DZA,Africa,Algeria,2020-07-06,15941.0,441.0,952.0,6.0,363.526,10.057,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-05,France,167650.0,0.0,29911.0,0.0,0.0,0.0
0,2020-07-05,China,84907.0,0.0,4651.0,0.0,0.0,0.0
0,2020-07-05,Italy,241771.0,0.0,34893.0,0.0,0.0,0.0
0,2020-07-05,Spain,251042.0,0.0,28411.0,0.0,0.0,0.0
0,2020-07-05,United States,2888777.0,0.0,130537.0,0.0,0.0,0.0
0,2020-07-05,World,11453025.0,0.0,536563.0,0.0,0.0,0.0
0,2020-07-05,United Kingdom,285891.0,0.0,44379.0,0.0,0.0,0.0
0,2020-07-05,Germany,196829.0,0.0,9028.0,0.0,0.0,0.0
0,2020-07-05,Iran,241087.0,0.0,11550.0,0.0,0.0,0.0
0,2020-07-05,Turkey,205712.0,0.0,5224.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-06,France,167650.0,166960.0,29911.0,29893.0,690.0,18.0
0,2020-07-06,China,84907.0,84871.0,4651.0,4641.0,36.0,10.0
0,2020-07-06,Italy,241771.0,241611.0,34893.0,34861.0,160.0,32.0
0,2020-07-05,Spain,251042.0,250545.0,28411.0,28385.0,497.0,26.0
0,2020-07-06,United States,2888777.0,2888635.0,130537.0,129947.0,142.0,590.0
0,2020-07-06,World,11453025.0,11418475.0,536563.0,533958.0,34550.0,2605.0
0,2020-07-06,United Kingdom,285891.0,285416.0,44379.0,44220.0,475.0,159.0
0,2020-07-06,Germany,196829.0,196554.0,9028.0,9016.0,275.0,12.0
0,2020-07-06,Iran,241087.0,240438.0,11550.0,11571.0,649.0,-21.0
0,2020-07-06,Turkey,205712.0,205758.0,5224.0,5225.0,-46.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")